In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
#from tqdm.auto import tqdm
from tqdm import tqdm
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in tqdm(os.listdir(maindir + '\\testing_modules')):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)
plt.close('All')

100%|██████████| 23/23 [00:06<00:00,  3.37it/s]


In [2]:
%matplotlib qt
import skimage.io as io
import xarray as xr
from collections import OrderedDict
import dask
from tqdm.dask import TqdmCallback
from sklearn.metrics.pairwise import euclidean_distances
from matplotlib import cm

In [3]:
def load_map_parameters(filename, filedir):
    arr = np.genfromtxt(f'{filedir}{filename}')
    return arr

In [4]:
scanid = 153432
base = '''D:\\Musterman_postdoc\\20240223_Musterman\\'''
map_filename = f'scan{scanid}_dexela_angle_rc.tif'
h5_filename = f'scan{scanid}_xrd.h5'

try:
    test = XRDMap.from_hdf(h5_filename, wd=f'{base}legacy\\')
except FileNotFoundError:
    test = XRDMap.from_image_stack(map_filename, wd=f'{base}legacy\\', energy=15, scanid=scanid, save_hdf=False)

test.set_calibration('scan153220_dexela_calibration.poni', filedir=f'{base}calibrations\\')
theta, i0, im, it = load_map_parameters(f'scan{test.scanid}_angle_rc_parameters.txt', filedir=test.wd)
theta /= 1000
energy = [10.2,] * len(theta)
test.map.dtype = np.float32

Loading images...

done!
Calibration performed under different settings. Adjusting calibration.


In [5]:
test.ai

Detector Detector	 Spline= None	 PixelSize= 7.480e-05, 7.480e-05 m
Wavelength= 8.265613e-11 m
SampleDetDist= 3.903584e-01 m	PONI= 1.087204e-01, 3.240324e-02 m	rot1=-0.910039  rot2=-0.002869  rot3=3.141593 rad
DirectBeamDist= 636.061 mm	Center: x=7146.912, y=1429.086 pix	Tilt= 52.142° tiltPlanRotation= -0.208° 𝛌= 0.827Å

In [17]:
orig_rot1 = test.ai.rot1
test.ai.rot1 = orig_rot1 + np.radians(-15)
print(np.degrees(test.ai.rot1))
test._del_arr()

-67.14140673862357


In [18]:
test.plot_image(test.tth_arr)

In [94]:
scanid = 153430
base = '''D:\\Musterman_postdoc\\20240223_Musterman\\'''
map_filename = f'scan{scanid}_dexela_energy_rc.tif'
h5_filename = f'scan{scanid}_xrd.h5'

try:
    test = XRDMap.from_hdf(h5_filename, wd=f'{base}proposals\\')
except FileNotFoundError:
    test = XRDMap.from_image_stack(map_filename, wd=f'{base}proposals\\', energy=15, scanid=scanid, save_hdf=False)

test.set_calibration('scan153220_dexela_calibration.poni', filedir=f'{base}calibrations\\')
energy, i0, im, it = load_map_parameters(f'scan{test.scanid}_energy_rc_parameters.txt', filedir=test.wd)
theta = [0,] * len(energy)
test.map.dtype = np.float32

Loading images...done!
Calibration performed under different settings. Adjusting calibration.


In [208]:
scanid = 153431
en = 10.2
th = np.linspace(-0.15, 0.15, 76)

map_filename = f'scan{scanid}_dexela_xrd.tif'
test = XRDMap.from_image_stack(map_filename, wd=f'{base}proposals\\', energy=en, scanid=scanid, save_hdf=False)
_, _, i0, i0_time, im, it  = load_map_parameters(f'scan{test.scanid}_map_parameters.txt', filedir=test.wd)
theta = th.copy()
energy = [en,] * len(theta)
scantype = 'fly'
test.map.dtype = np.float32
test.set_calibration('scan153220_dexela_calibration.poni', filedir=f'{base}calibrations\\')

Loading images...done!
Calibration performed under different settings. Adjusting calibration.


In [209]:
sat_mask = test.map.images == 16383
print(f'{np.sum(sat_mask)} saturated pixels found.')

76 saturated pixels found.


In [210]:
dark_dir = f'{base}dark_fields\\'
dark_field = io.imread(f'{dark_dir}scan153332_dexela_1x1_hr_0.1sec.tif')
#dark_field = io.imread(f'{dark_dir}scan153476_dexela_1x1_ln_0.1sec.tif')
#scaled_df = dark_field * np.median(test.map.min_image) / np.median(dark_field)
test.map.correct_dark_field(dark_field=dark_field)

Correcting dark-field...done!


In [211]:
#det_sets = '_hr_1x1_'
#flat_dir = '''D:\\Musterman_postdoc\\20240125_Musterman\\flat_fields\\'''
#flat_paths = np.array(os.listdir(flat_dir))
#flat_mask = [det_sets in x for x in flat_paths]
#flat_field = io.imread(f'{flat_dir}{flat_paths[flat_mask][0]}')
#test.map.correct_flat_field(flat_field=flat_field)

In [212]:
test.map.correct_outliers()
test.map.normalize_scaler(scaler_arr=i0.reshape(1, -1))

Finding and correcting image outliers...done!
Normalize image scalers...done!


In [213]:
test.map.apply_lorentz_correction()
test.map.apply_polarization_correction()
test.map.apply_solidangle_correction()

Applying Lorentz correction...done!
Applying X-ray polarization correction...done!
Applying solid angle correction...done!


In [214]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.load_phase('AMCSD\\Corundum_0009327.cif', filedir=cif_dir, phase_name="corundum")
test.load_phase('AMCSD\\Silicon_0011244.cif', filedir=cif_dir, phase_name="silicon")

absorptions = []

exp_dict = {
    'attenuation_length' : 0,
    'mode' : 'transmission',
    'thickness' : 200, # microns
    'theta' : 0
}

for i, (en, th) in enumerate(zip(energy, theta)):
    test.energy = en

    exp_dict['attenuation_length'] = test.phases['corundum'].absorption_length(en=en * 1e3)
    #|exp_dict['attenuation_length'] = test.phases['silicon'].absorption_length(en=en * 1e3)
    exp_dict['theta'] = th
    test.map.apply_absorption_correction(exp_dict=exp_dict, apply=False)
    absorptions.append(test.map.absorption_correction)

for i, absorb in enumerate(absorptions):
    test.map.images[0, i] /= absorb

test.map.corrections['absorption'] = True

In [ ]:
'''t = 200
tth, chi = 20, 90
theta = 20

tth = np.radians(tth)
chi = np.radians(chi)
theta = np.radians(theta)

x = t * ((np.cos(theta) / np.cos(tth)) * (1 + np.cos(chi)) - np.cos(chi) / (np.sin(tth) * np.tan(theta)))
x = t * ((np.cos(theta) / np.cos(tth)) + np.cos(chi) * (np.cos(theta) / np.cos(tth) - 1 / (np.sin(tth) * np.tan(theta))))
x = t * (1 / (np.cos(theta) * np.cos(tth)))
x = (t / np.cos(tth)) * ((1 / np.cos(theta)) + np.cos(chi) * ((np.tan(tth) * np.sin(theta)) / (1 - np.tan(tth))))
x = t * ((np.cos(theta) * (1 - np.tan(tth))) / (np.tan(tth) - np.tan(theta)))
x = t * (1 / (np.sin(tth) * np.cos(tth) * ((1 / np.tan(tth) - np.tan(theta)))))
x = t * (np.sqrt(1 + (np.cos(chi) / np.tan(tth))**2) / (np.cos(theta) * ((np.cos(chi) / np.tan(tth)) - np.tan(theta))))
x = t * ((1 / (np.cos(tth) * np.cos(theta))) + ((np.cos(chi) * np.tan(theta)) / ((1 / np.tan(tth)) - np.tan(theta))))
x = t * ((1 / (np.cos(tth) * np.cos(theta))) + (np.cos(chi) * np.tan(theta)) / (np.cos(tth) * np.cos(theta) * ((1 / np.tan(tth)) - np.tan(theta))))
x = t * (1 / ((1 / (np.cos(theta) * np.cos(tth) * np.tan(tth))) * ((1 / np.tan(tth)) - np.cos(chi) * np.tan(theta))))
x = t / (np.cos(tth) * np.cos(theta))
print(x)

x = (t / (np.cos(theta))) / ((1 / np.tan(tth)) - (np.cos(chi) * np.tan(theta)))
y = x / (np.tan(tth))
d = np.sqrt(x**2 + y**2)
print(x, y, d)''';

In [ ]:
#test.map.estimate_background(method='bruckner', binning=8, min_prominence=0.1)

In [ ]:
#test.map.remove_background()

In [215]:
test.map.rescale_images(arr_min=0)
test.map.finalize_images()

Caution: Images not corrected for:
	flat_field
	pixel_defects
	pixel_distortions
	polar_calibration
	background
Cleaning and updating image information...
Diffraction map size is 1.691 GB.
No hdf file specified. Images will not be saved.


In [221]:
test.scanid

153431

In [217]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

rc = np.max(test.map.images, axis=(0, 2, 3))
max_rc = np.argmax(rc)
max_rc = int(np.round(arbitrary_center_of_mass(rc, range(test.map.num_images))[0], 0))
test.energy = energy[max_rc]
max_indices = np.unravel_index(np.argmax(test.map.images[0, max_rc]), test.map.image_shape)
q1 = test.q_arr[:, *max_indices]

th = np.radians(theta[max_rc])
Ry = np.array([[np.cos(th), 0, -np.sin(th)],
               [0, 1, 0],
               [np.sin(th), 0, np.cos(th)]])
q1 = Ry @ q1

if theta[0] - theta[1] != 0:
    ax.plot(theta, rc)
    ax.scatter(theta[max_rc], rc[max_rc], s=10, c='r')
else:
    ax.plot(energy, rc)
    ax.scatter(energy[max_rc], rc[max_rc], s=10, c='r')

plt.show()

In [222]:
q1

array([-3.03970749,  0.00702176, -0.9915089 ])

In [223]:
test.phases['corundum'].HKL(q1)

array([-1.99569668,  0.00531618, -2.04950508])

In [224]:
max_indices

(1433, 1940)

In [226]:
q_image_list = []
qx_min, qx_max = np.nan, np.nan
qy_min, qy_max = np.nan, np.nan
qz_min, qz_max = np.nan, np.nan


for i, (en, th) in enumerate(zip(energy, theta)):
    test.energy = en
    th = np.radians(th)

    image = test.map.images[0, i]

    q_mask = np.ones_like(test.map.mask, dtype=np.bool_)
    scale = 0.05

    Ry = np.array([[np.cos(th), 0, -np.sin(th)],
                   [0, 1, 0],
                   [np.sin(th), 0, np.cos(th)]])

    q = test.q_arr.copy()
    q = (q.T @ Ry.T).T # I don't like this

    for i in range(len(q1)):
        mask = (((q1[i] - scale < q[i]))
                & (q[i] < (q1[i] + scale)))
        q_mask *= mask

    q_reduced = q[:, q_mask]
    int_reduced = image[q_mask]
    q_image = np.vstack([q_reduced, int_reduced])
    
    qx_min = np.nanmin([*q_reduced[0], qx_min])
    qx_max = np.nanmax([*q_reduced[0], qx_max])
    qy_min = np.nanmin([*q_reduced[1], qy_min])
    qy_max = np.nanmax([*q_reduced[1], qy_max])
    qz_min = np.nanmin([*q_reduced[2], qz_min])
    qz_max = np.nanmax([*q_reduced[2], qz_max])

    q_image_list.append(q_image)

In [231]:
qx, qy, qz, q_int = [], [], [], []
for q_image in q_image_list:
    int_mask = q_image[-1] > -100
    qx.extend(q_image[0][int_mask])
    qy.extend(q_image[1][int_mask])
    qz.extend(q_image[2][int_mask])
    q_int.extend(q_image[-1][int_mask])

q_spot = np.asarray([qx, qy, qz, q_int])

In [232]:
hdf_file = 'D:\\Musterman_postdoc\\20240223_Musterman\\20240223_sapphire_all.h5'

with h5py.File(hdf_file, 'a') as f:
    base_grp = f.require_group('sapphire')
    angle_rc = base_grp.require_group('angle_rc')
    energy_rc = base_grp.require_group('energy_rc')
    fly_angle_rc = base_grp.require_group('fly_angle_rc')

    group = fly_angle_rc

    # Check for and delete dataset if it exists
    if f'scan{test.scanid}' in group.keys():
        print('Found existing dataset. Deleting and rewriting...')
        del group[f'scan{test.scanid}']

    # Make new dataset
    new_data = group.require_dataset(f'scan{test.scanid}',
                                     data=q_spot,
                                     shape=q_spot.shape,
                                     dtype=q_spot.dtype)
    new_data.attrs['labels'] = ['qx', 'qy', 'qz', 'q_int']
    new_data.attrs['theta'] = theta
    new_data.attrs['energy'] = energy
    q_cen = arbitrary_center_of_mass(q_spot[-1], *q_spot[:3])
    new_data.attrs['center_of_mass'] = q_cen

    print(np.array(group.keys()))


Found existing dataset. Deleting and rewriting...
<KeysViewHDF5 ['scan153365', 'scan153366', 'scan153371', 'scan153374', 'scan153377', 'scan153383', 'scan153387', 'scan153393', 'scan153397', 'scan153402', 'scan153408', 'scan153411', 'scan153428', 'scan153431']>


In [ ]:
energy_scanids =  [153334, 153344, 153350, 153359, 153381, 153386, 153391, 153396, 153400, 153405, 153410, 153415, 153426, 153430]
angle_scanids = [153362, 153367, 153372, 153375, 153378, 153384, 153388, 153394, 153398, 153403, 153409, 153413, 153416, 153429, 153432]
#angle_scanids = [153384, 153398, 153403, 153409, 153413, 153416, 153429, 153432]
energies = [20.25, 20.375, 19.225, 18.375, 14.55, 18, 18, 18, 18, 18, 18, 18, 18, 10.125, 10.20]
#energies = [18, 18, 18, 18, 18, 18, 10.125, 10.20]
thetas = [0, 0, 0, 0, 5, 5.5, 12.25, 13.625, 14.2, 15.55, 12.35, 12.35, 0, 0, 0]
fly_scanids = [153365, 153366, 153371, 153374, 153377, 153383, 153387, 153393, 153397, 153402, 153408, 153411, 153428, 153431]
fly_energies = [20.25, 20.375, 19.225, 18.3275, 14.550, 18, 18, 18, 18, 18, 18, 18, 10.125, 10.2]
fly_thetas = [np.linspace(-0.1, 0.1, 51),
              np.linspace(-0.1, 0.1, 51),
              np.linspace(-0.1, 0.1, 51),
              np.linspace(-0.1, 0.1, 51),
              np.linspace(-0.1, 0.1, 51),
              np.linspace(4.9, 5.1, 51),
              np.linspace(5.4, 5.6, 51),
              np.linspace(12.15, 12.45, 76),
              np.linspace(13.525, 13.725, 51),
              np.linspace(14.1, 14.4, 76),
              np.linspace(15.450, 15.75, 76),
              np.linspace(12.25, 12.55, 76),
              np.linspace(-0.1, 0.1, 51),
              np.linspace(-0.15, 0.15, 76)]


base = 'D:\\Musterman_postdoc\\20240223_Musterman\\'
dark_dir = f'{base}dark_fields\\'
cif_dir = 'C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'
hdf_file = 'D:\\Musterman_postdoc\\20240223_Musterman\\20240223_sapphire_0.1.h5'

t_start = ttime.monotonic()
#for scanid, en, th in zip(angle_scanids, energies, energies):
#for scanid, en, th in zip(energy_scanids, thetas, thetas):
for scanid, en, th in zip(fly_scanids, fly_energies, fly_thetas):
    t0 = ttime.monotonic()
    # Load images
    try: # angle rc
        map_filename = f'scan{scanid}_dexela_angle_rc.tif'
        test = XRDMap.from_image_stack(map_filename, wd=f'{base}legacy\\', energy=15, scanid=scanid, save_hdf=False)
        theta, i0, im, it = load_map_parameters(f'scan{test.scanid}_angle_rc_parameters.txt', filedir=test.wd)
        theta /= 1000
        energy = [en,] * len(theta)
        scantype = 'angle'
    except FileNotFoundError:
        try: # energy rc
            map_filename = f'scan{scanid}_dexela_energy_rc.tif'
            test = XRDMap.from_image_stack(map_filename, wd=f'{base}proposals\\', energy=15, scanid=scanid, save_hdf=False)
            energy, i0, im, it = load_map_parameters(f'scan{test.scanid}_energy_rc_parameters.txt', filedir=test.wd)
            theta = [th,] * len(energy)
            scantype = 'energy'
        except FileNotFoundError: # fly angle rc
            map_filename = f'scan{scanid}_dexela_xrd.tif'
            test = XRDMap.from_image_stack(map_filename, wd=f'{base}proposals\\', energy=en, scanid=scanid, save_hdf=False)
            _, _, i0, i0_time, im, it  = load_map_parameters(f'scan{test.scanid}_map_parameters.txt', filedir=test.wd)
            theta = th.copy()
            energy = [en,] * len(theta)
            scantype = 'fly'

    test.set_calibration('scan153220_dexela_calibration.poni', filedir=f'{base}calibrations\\')
    test.map.dtype = np.float32

    sat_mask = test.map.images == 16383
    print(f'{np.sum(sat_mask)} saturated pixels found.')

    # Image corrections
    dark_field = io.imread(f'{dark_dir}scan153332_dexela_1x1_hr_0.1sec.tif')
    test.map.correct_dark_field(dark_field=dark_field)
    test.map.correct_outliers()
    test.map.normalize_scaler(scaler_arr=i0.reshape(1, -1))
    test.map.apply_lorentz_correction()
    test.map.apply_polarization_correction()
    test.map.apply_solidangle_correction()

    # Figure out absorption corrections
    test.load_phase('AMCSD\\Corundum_0009327.cif', filedir=cif_dir, phase_name="corundum")
    corundum = test.phases['corundum']

    absorptions = []
    exp_dict = {
        'attenuation_length' : 0,
        'mode' : 'transmission',
        'thickness' : 200, # microns
        'theta' : 0
    }

    for i, (en, th) in enumerate(zip(energy, theta)):
        test.energy = en

        exp_dict['attenuation_length'] = test.phases['corundum'].absorption_length(en=en * 1e3)
        exp_dict['theta'] = th
        test.map.apply_absorption_correction(exp_dict=exp_dict, apply=False)
        absorptions.append(test.map.absorption_correction)

    for i, absorb in enumerate(absorptions):
        test.map.images[0, i] /= absorb
    test.map.corrections['absorption'] = True

    test.map.rescale_images(arr_min=0)
    test.map.finalize_images()

    # Find where to cut out q-space
    rc = np.max(test.map.images, axis=(0, 2, 3))
    test.energy = energy[np.argmax(rc)]
    max_indices = np.unravel_index(np.argmax(test.map.images[0, np.argmax(rc)]), test.map.image_shape)
    q1 = test.q_arr[:, *max_indices]

    th = np.radians(theta[np.argmax(rc)])
    Ry = np.array([[np.cos(th), 0, -np.sin(th)],
                [0, 1, 0],
                [np.sin(th), 0, np.cos(th)]])
    q1 = Ry @ q1

    # Cut out the correct q-space
    q_image_list = []
    qx_min, qx_max = np.nan, np.nan
    qy_min, qy_max = np.nan, np.nan
    qz_min, qz_max = np.nan, np.nan

    for i, (en, th) in enumerate(zip(energy, theta)):
        test.energy = en
        th = np.radians(th)

        image = test.map.images[0, i]

        q_mask = np.ones_like(test.map.mask, dtype=np.bool_)
        scale = 0.05

        Ry = np.array([[np.cos(th), 0, -np.sin(th)],
                    [0, 1, 0],
                    [np.sin(th), 0, np.cos(th)]])

        q = test.q_arr.copy()
        q = (q.T @ Ry.T).T # I don't like this

        for i in range(len(q1)):
            mask = (((q1[i] - scale < q[i]))
                    & (q[i] < (q1[i] + scale)))
            q_mask *= mask

        q_reduced = q[:, q_mask]
        int_reduced = image[q_mask]
        q_image = np.vstack([q_reduced, int_reduced])
        
        qx_min = np.nanmin([*q_reduced[0], qx_min])
        qx_max = np.nanmax([*q_reduced[0], qx_max])
        qy_min = np.nanmin([*q_reduced[1], qy_min])
        qy_max = np.nanmax([*q_reduced[1], qy_max])
        qz_min = np.nanmin([*q_reduced[2], qz_min])
        qz_max = np.nanmax([*q_reduced[2], qz_max])

        q_image_list.append(q_image)

    for cut_off in [-100, 0.1, 1]:
        if cut_off == -100:
            hdf_ext = 'all'
        else:
            hdf_ext = str(cut_off)
        hdf_file = f'D:\\Musterman_postdoc\\20240223_Musterman\\20240223_sapphire_{hdf_ext}.h5'

        # Consolidate and trim data
        qx, qy, qz, q_int = [], [], [], []
        for q_image in q_image_list:
            int_mask = q_image[-1] > cut_off
            qx.extend(q_image[0][int_mask])
            qy.extend(q_image[1][int_mask])
            qz.extend(q_image[2][int_mask])
            q_int.extend(q_image[-1][int_mask])

        q_spot = np.asarray([qx, qy, qz, q_int])

        # Save data
        with h5py.File(hdf_file, 'a') as f:
            base_grp = f.require_group('sapphire')
            angle_rc = base_grp.require_group('angle_rc')
            energy_rc = base_grp.require_group('energy_rc')
            fly_angle_rc = base_grp.require_group('fly_angle_rc')

            if scantype == 'angle':
                group = angle_rc
            elif scantype == 'energy':
                group = energy_rc
            elif scantype == 'fly':
                group = fly_angle_rc

            new_data = group.require_dataset(f'scan{test.scanid}',
                                            data=q_spot,
                                            shape=q_spot.shape,
                                            dtype=q_spot.dtype)
            new_data.attrs['labels'] = ['qx', 'qy', 'qz', 'q_int']
            new_data.attrs['theta'] = theta
            new_data.attrs['energy'] = energy

            print(list(group.keys()))

    # Time estimates
    if scantype == 'angle':
        scanlist = angle_scanids
    elif scantype == 'energy':
        scanlist = energy_scanids
    elif scantype == 'fly':
        scanlist = fly_scanids

    tf = ttime.monotonic()
    dt = tf - t0
    total_dt = tf - t_start
    print_time = ttime.strftime('%H:%M:%S', ttime.gmtime(tf-t0))
    print(f'Scan {test.scanid} took {print_time}.')
    num_scans_finished = np.where(np.array(scanlist) == test.scanid)[0][0] + 1
    num_scans_remaining = len(scanlist) - num_scans_finished
    avg_scan_time = total_dt / num_scans_finished
    est_time_remaining = avg_scan_time * num_scans_remaining
    print_time_remain = ttime.strftime('%H:%M:%S', ttime.gmtime(est_time_remaining))
    print_time_fin = ttime.strftime('%H:%M:%S', ttime.localtime(ttime.mktime(ttime.localtime()) + est_time_remaining))
    print(f'{num_scans_remaining} scans remaining. Estimated {print_time_remain} time remaining completing at {print_time_fin}')

    print('#' * 72)

Loading images...Loading images...Loading images...done!
Calibration performed under different settings. Adjusting calibration.
168 saturated pixels found.
Correcting dark-field...done!
Finding and correcting image outliers...done!
Normalize image scalers...done!
Applying Lorentz correction...done!
Applying X-ray polarization correction...done!
Applying solid angle correction...done!
Caution: Images not corrected for:
	flat_field
	pixel_defects
	pixel_distortions
	polar_calibration
	background
Cleaning and updating image information...
Diffraction map size is 1.135 GB.
No hdf file specified. Images will not be saved.
['scan153365']
['scan153365']
['scan153365']
Scan 153365 took 00:01:56.
13 scans remaining. Estimated 00:25:14 time remaining completing at 10:22:38
########################################################################
Loading images...Loading images...Loading images...done!
Calibration performed under different settings. Adjusting calibration.
51 saturated pixels found

In [ ]:
with h5py.File(f'{base}20240223_sapphire_1.h5', 'a') as f:
    base_grp = f['sapphire']
    scan_grp_keys = base_grp.keys()

    for scan_key in scan_grp_keys:
        for scanid in base_grp[scan_key].keys():
            q_spot = base_grp[scan_key][scanid][:]
            q_cen = arbitrary_center_of_mass(q_spot[-1], *q_spot[:3])
            base_grp[scan_key][scanid].attrs['center_of_mass'] = q_cen

In [8]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.load_phase('AMCSD\\Corundum_0009327.cif', filedir=cif_dir, phase_name="corundum")
test.load_phase('AMCSD\\Silicon_0011244.cif', filedir=cif_dir, phase_name="silicon")

In [45]:
base = 'D:\\Musterman_postdoc\\20240223_Musterman\\'

refls = {'scanid' : [],
         'q_cen' : [],
         'guess_hkl' : [],
         'q_val' : []}

with h5py.File(f'{base}20240223_sapphire_1.h5', 'r') as f:
    base_grp = f['sapphire']
    scan_grp_keys = base_grp.keys()

    for scan_key in scan_grp_keys:
        for scanid in base_grp[scan_key].keys():
            q_cen = base_grp[scan_key][scanid].attrs['center_of_mass']
            guess_hkl = np.round(test.phases['corundum'].HKL(q_cen)).astype(np.int32)

            refls['scanid'].append(int(scanid[-6:]))
            refls['q_cen'].append(q_cen)
            refls['guess_hkl'].append(guess_hkl)
            refls['q_val'].append(base_grp[scan_key][scanid][:])

In [305]:
ind = np.where(np.array(refls['scanid']) == 153393)[0][0]

In [306]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

q_plot = refls['q_val'][ind]

qx_min, qx_max = np.min(q_plot[0]), np.max(q_plot[0])
qy_min, qy_max = np.min(q_plot[1]), np.max(q_plot[1])
qz_min, qz_max = np.min(q_plot[2]), np.max(q_plot[2])

int_mask = q_plot[-1] > 0.5

#ax.scatter(*q_plot[:3][int_mask], c=q_plot[-1][int_mask], s=1, alpha=0.5)
ax.scatter(q_plot[0][int_mask],
           q_plot[1][int_mask],
           q_plot[2][int_mask],
           c=q_plot[-1][int_mask], s=1, alpha=0.1)

ax.set_xlim(qx_min, qx_max)
ax.set_ylim(qy_min, qy_max)
ax.set_zlim(qz_min, qz_max)

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [307]:
from scipy.interpolate import griddata

q_spot = refls['q_val'][ind]

all_qx = q_spot[0]
all_qy = q_spot[1]
all_qz = q_spot[2]
all_int = q_spot[3]

# Find bounds
x_min = np.min(all_qx)
x_max = np.max(all_qx)
y_min = np.min(all_qy)
y_max = np.max(all_qy)
z_min = np.min(all_qz)
z_max = np.max(all_qz)

# Generate q-space grid
gridstep = 0.001
xx = np.linspace(x_min, x_max, int((x_max - x_min) / gridstep))
yy = np.linspace(y_min, y_max, int((y_max - y_min) / gridstep))
zz = np.linspace(z_min, z_max, int((z_max - z_min) / gridstep))

grid = np.array(np.meshgrid(xx, yy, zz))
grid = grid.reshape(3, grid.size // 3).T

points = np.array([all_qx, all_qy, all_qz]).T

int_grid = griddata(points, all_int, grid, method='nearest')
int_grid = int_grid.reshape(yy.shape[0], xx.shape[0], zz.shape[0]).T

In [308]:
from skimage import measure
from matplotlib.ticker import FormatStrFormatter
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

spacing = 0.005

vol = int_grid
#vol = GaussianFunctions.func_3d(grid, *popt).reshape(*int_grid.shape)
#vol = GaussianFunctions.func_3d(np.array(np.meshgrid(xx, yy, zz)), *popt).T.reshape(*int_grid.shape)

iso_vals = np.linspace(np.min(int_grid), np.max(int_grid), 7)[1:-1]
iso_vals = [2.5, 5, 10, 20, 40, 80]

for i, iso_val in enumerate(iso_vals):

    #vol = int_grid
    #iso_val = 10000
    #spacing = 0.001
    color = cm.viridis(i / len(iso_vals))
    verts, faces, _, _ = measure.marching_cubes(vol, iso_val)

    # I have no idea how the axes got so mixed up...
    plot_x = np.interp(verts[:, 1], range(len(xx)), xx)
    plot_y = np.interp(verts[:, 2], range(len(yy)), yy)
    plot_z = np.interp(verts[:, 0], range(len(zz)), zz)

    ax.plot_trisurf(plot_x, plot_y, plot_z, triangles=faces, alpha=0.1, color=color)

ax.set_xlim(np.min(all_qx), np.max(all_qx))
ax.set_ylim(np.min(all_qy), np.max(all_qy))
ax.set_zlim(np.min(all_qz), np.max(all_qz))

ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [31]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

for q_cen in refls['q_cen']:
    ax.scatter(*q_cen, s=1, c='k')
ax.scatter(0, 0, 0, s=10, c='r')


ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()

In [724]:
q_uniq = []
q_arr = np.asarray(list(q_centers.values()))

for q_i in q_arr:
    if len(q_uniq) == 0:
        q_uniq.append(q_i)
    else:
        q_diff = np.array(q_uniq) - q_i
        if np.all(np.any(np.abs(q_diff) > 0.05, axis=1)):
            q_uniq.append(q_i)

In [64]:
fit_ori, fit_min = iterative_dictionary_indexing(q_uniq,
                                                 test.phases['corundum'],
                                                 [np.min(test.tth_arr), np.max(test.tth_arr)],
                                                 cut_off=0.05,
                                                 start_angle=10,
                                                 angle_resolution=0.01, 
                                                 euler_bounds=[[-180, 180], [0, 180], [-180, 180]])

In [65]:
fit_ori

[(-150, 120, 150),
 (-150, 120, 150),
 (-95.0, 67.5, -22.5),
 (-95.0, 67.5, -22.5),
 (-24.375, 111.875, 158.125),
 (-144.6875, 112.1875, 157.8125),
 (-95.3125, 67.65625, -22.1875),
 (-95.3125, 67.65625, -22.1875),
 (-95.3125, 67.65625, -22.1875),
 (-95.3125, 67.65625, -22.20703125),
 (-95.3125, 67.65625, -22.20703125)]

In [223]:
all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['corundum'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))
#plot_qs = euler_rotation(all_qs, *-np.array(fit_ori[-1]))
plot_qs = euler_rotation(all_qs, 0, 0, 0)

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs/20)
ax.scatter(0, 0, 0, c='k', s=10)

for q_cen in q_centers.values():
    ax.scatter(*q_cen, s=1, c='r')

lat_const = -test.phases['corundum'].Q([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
ax.quiver([0, 0, 0], [0 ,0, 0], [0, 0, 0], *lat_const.T, color=[[1, 0, 0], [0, 1, 0], [0, 0, 1]])

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()

In [12]:
def iterative_dictionary_indexing(spot_qs, Phase, tth_range, cut_off=0.1, start_angle=10, angle_resolution=0.001,
                                  euler_bounds=[[-180, 180], [0, 180], [-180, 180]]):
    from itertools import product

    all_hkls, all_qs, all_fs = generate_reciprocal_lattice(Phase, tth_range=tth_range)

    dist = euclidean_distances(all_qs)
    min_q = np.min(dist[dist > 0])

    step = start_angle
    #print(f'Finding orientations with {step} deg resolution...')
    phi1_list = np.arange(*euler_bounds[0], step)
    PHI_list = np.arange(*euler_bounds[1], step)
    phi2_list = np.arange(*euler_bounds[2], step)
    orientations = list(product(phi1_list, PHI_list, phi2_list))

    fit_ori = []
    fit_min = []
    ITERATE = True
    while ITERATE:
        step /= 2 # half the resolution each time
        if step <= angle_resolution:
            ITERATE = False

        #print(f'Evaluating the current Euler space...')
        min_list = []
        for orientation in orientations:
            dist = euclidean_distances(spot_qs, euler_rotation(all_qs, *orientation))
            min_list.append(np.sum(np.min(dist, axis=1)**2))
        
        fit_min.append(np.min(min_list))
        fit_ori.append(orientations[np.argmin(min_list)])
        
        min_mask = min_list < cut_off * (np.max(min_list) - np.min(min_list)) + np.min(min_list)
        best_orientations = np.asarray(orientations)[min_mask]

        #print(f'Finding new orientations with {step:.4f} deg resolution...')
        new_orientations = []
        for orientation in best_orientations:
            phi1, PHI, phi2 = orientation
            new_phi1 = [phi1 - step, phi1, phi1 + step]
            new_PHI = [PHI - step, PHI, PHI + step]
            new_phi2 = [phi2 - step, phi2, phi2 + step]

            sub_orientations = product(new_phi1, new_PHI, new_phi2)

            for sub_orientation in sub_orientations:
                if sub_orientation not in new_orientations:
                    new_orientations.append(sub_orientation)
            
            orientations = new_orientations

    #print(f'Evaluating the last Euler space...')
    min_list = []
    for orientation in orientations:
        dist = euclidean_distances(spot_qs, euler_rotation(all_qs, *orientation))
        min_list.append(np.sum(np.min(dist, axis=1)**2))
    
    fit_min.append(np.min(min_list))
    fit_ori.append(orientations[np.argmin(min_list)])

    return fit_ori, fit_min

In [1306]:
rot1, rot2, rot3 = -test.ai.rot1, -test.ai.rot2, test.ai.rot3

Rx = np.asarray([[1, 0, 0],
                 [0, np.cos(rot1), -np.sin(rot1)],
                 [0, np.sin(rot1), np.cos(rot1)]])

Ry = np.asarray([[np.cos(rot2), 0, np.sin(rot2)],
                 [0, 1, 0],
                 [-np.sin(rot2), 0, np.cos(rot2)]])

Rz = np.asarray([[np.cos(rot3), -np.sin(rot3), 0],
                 [np.sin(rot3), np.cos(rot3), 0],
                 [0, 0, 1]])

p1 = 0 * test.ai.pixel1 - test.ai.poni1 + (test.ai.pixel1 / 2) # half pixel offset
p2 = 0 * test.ai.pixel2 - test.ai.poni2 + (test.ai.pixel2 / 2) # half pixel offset
p3 = test.ai.dist
pixel = np.array([p1, p2, p3])
R = Rz @ Ry @ Rx
t1, t2, t3 = R @ pixel
xyz = np.array([t3, t1, t2])
xyz

array([0.2159456 , 0.10880118, 0.32953855])

In [1223]:
t = 500e-6
t = 0
Rt = np.array([[*R[0], 0],
               [*R[1], 0],
               [*R[2], t],
               [0, 0, 0, 1]])

In [1224]:
Rt @ [test.ai.poni1, test.ai.poni2, test.ai.dist, 1]

array([-0.1094807,  0.2883125,  0.264839 ,  1.       ])

In [10]:
unstrained = LatticeParameters.from_Phase(test.phases['corundum'])

In [238]:
unstrained.Amat

array([[ 4.757     , -2.3785    ,  0.        ],
       [ 0.        ,  4.11968285,  0.        ],
       [ 0.        ,  0.        , 12.9877    ]])

In [14]:
unstrained.Bmat

array([[ 0.24273713,  0.12136857, -0.        ],
       [ 0.        ,  0.21021652, -0.        ],
       [ 0.        ,  0.        ,  0.07699593]])

In [16]:
unstrained.b1

array([0.24273713, 0.        , 0.        ])

In [19]:
lp = [unstrained.a,
      unstrained.b,
      unstrained.c,
      unstrained.alpha,
      unstrained.beta,
      unstrained.gamma]

rp = [unstrained.a_star,
      unstrained.b_star,
      unstrained.c_star,
      unstrained.alpha_star,
      unstrained.beta_star,
      unstrained.gamma_star]

In [30]:
np.degrees(unstrained.gamma)

119.99999999999999

In [28]:
strained.a3 @ strained.b3

0.9999909821469473

In [20]:
rp

[0.2427371323059179,
 0.2427371323059179,
 0.07699592691546615,
 1.5707963267948968,
 1.5707963267948968,
 1.0471975511965979]

In [22]:
lp

[4.757,
 4.757,
 12.9877,
 1.5707963267948966,
 1.5707963267948966,
 2.0943951023931953]

In [23]:
LatticeParameters.convert_lat_const(*rp)

(4.757,
 4.757,
 12.987700000000002,
 1.5707963267948966,
 1.5707963267948966,
 2.0943951023931953)

In [56]:
scanlist = [153334, 153344, 153350, 153354, 153359, 153426, 153430]
#scanlist = [153362, 153367, 153372, 153375, 153378, 153429, 153432]
#scanlist = [153365, 153366, 153371, 153374, 153377, 153428, 153431]
#scanlist = [153381, 153386, 153391, 153396, 153400, 153405]
#scanlist = [153384, 153388, 153394, 153398, 153403, 153409]
#scanlist = [153383, 153387, 153393, 153397, 153402, 153408]
exp_mask = np.array([d in scanlist for d in refls['scanid']])

In [57]:
# Fitting reflections
from scipy import linalg
from scipy.spatial.transform import Rotation
I = np.asarray([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

x, res, rnk, s = linalg.lstsq(np.array(refls['guess_hkl'])[exp_mask], np.array(refls['q_cen'])[exp_mask])
UBmat = x.T / (2 * np.pi) # why transpose???
strained = LatticeParameters.from_UBmat(UBmat)
old_strained = LatticeParameters.from_Bmat(UBmat) 
Tij = np.dot(strained.Bmat, np.linalg.inv(unstrained.Bmat))
eij = 0.5 * (Tij + Tij.T) - I
U = UBmat @ linalg.inv(strained.Bmat)
print((eij - np.trace(eij) * I) * 1e3)
print(np.trace(eij) * 1e3)
print(U)
print(Rotation.from_matrix(U).as_euler('xzx'))

[[-5.81592641  1.1637518   1.35684899]
 [ 1.1637518  -0.57583109 -0.34950231]
 [ 1.35684899 -0.34950231 -1.18956106]]
3.7906592821976837
[[ 0.99997592 -0.00155771 -0.00676245]
 [ 0.00147959  0.9999323  -0.01154107]
 [ 0.00677997  0.01153079  0.99991053]]
[-1.34439841  0.0069396   1.35593474]


In [337]:
# Fitting reflections
from scipy import linalg
I = np.asarray([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

x, res, rnk, s = linalg.lstsq(refls['guess_hkl'], refls['q_cen'])
UBmat = x.T / (2 * np.pi) # why transpose???
strained = LatticeParameters.from_UBmat(UBmat)
old_strained = LatticeParameters.from_Bmat(UBmat) 
Tij = np.dot(strained.Bmat, np.linalg.inv(unstrained.Bmat))
eij = 0.5 * (Tij + Tij.T) - I
new_eij = U @ eij @ U.T
print((eij - np.trace(eij) * I) * 1e3)
print(np.trace(eij) * 1e3)

[[-3.96386753 -1.19938008  0.3512535 ]
 [-1.19938008  0.2711375   2.45731339]
 [ 0.3512535   2.45731339 -1.44222133]]
2.5674756801311593


In [335]:
new_eij = U.T @ eij @ U
(new_eij - np.trace(new_eij) * I)
print((new_eij - np.trace(new_eij) * I) * 1e3)
print(np.trace(new_eij) * 1e3)

[[-3.95586312 -1.18309112  0.37828077]
 [-1.18309112  0.31803431  2.44840238]
 [ 0.37828077  2.44840238 -1.49712255]]
2.5674756801311576


In [334]:
print((eij - np.trace(eij) * I) * 1e3)
print(np.trace(eij) * 1e3)

[[-3.96386753 -1.19938008  0.3512535 ]
 [-1.19938008  0.2711375   2.45731339]
 [ 0.3512535   2.45731339 -1.44222133]]
2.5674756801311593


In [249]:
r, v = linalg.polar(UBmat, side='left')

In [250]:
v

array([[0.26540713, 0.05416259, 0.00189336],
       [0.05416259, 0.20356794, 0.00177317],
       [0.00189336, 0.00177317, 0.07711504]])

In [251]:
r

array([[ 0.96600124,  0.25843659, -0.00722052],
       [-0.25849452,  0.96597298, -0.00876298],
       [ 0.00471015,  0.01033151,  0.99993554]])

In [252]:
U = UBmat @ linalg.inv(strained.Bmat)
U @ U.T # This should not have worked
U

array([[ 0.99997369,  0.00113096, -0.00716476],
       [-0.00120345,  0.99994805, -0.01012168],
       [ 0.00715294,  0.01013003,  0.99992311]])

In [240]:
eij *1e3

array([[-1.39639185, -1.19938008,  0.3512535 ],
       [-1.19938008,  2.83861318,  2.45731339],
       [ 0.3512535 ,  2.45731339,  1.12525435]])

In [243]:
np.trace(eij) * 1e3

2.5674756801311593

In [244]:
(eij - np.trace(eij) * I) * 1e6

array([[-3963.86753285, -1199.38007712,   351.253497  ],
       [-1199.38007712,   271.13750213,  2457.31338969],
       [  351.253497  ,  2457.31338969, -1442.22132954]])

In [247]:
new_eij = U @ eij @ U.T
new_eij

array([[-0.00140404, -0.0012155 ,  0.00032362],
       [-0.0012155 ,  0.00279159,  0.00246511],
       [ 0.00032362,  0.00246511,  0.00117993]])

In [248]:
(new_eij - np.trace(new_eij) * I) * 1e6

array([[-3971.51847768, -1215.4977806 ,   323.62357906],
       [-1215.4977806 ,   224.10934522,  2465.10988514],
       [  323.62357906,  2465.10988514, -1387.5422278 ]])

In [947]:
b1, b2, b3 = x
b1_mag = linalg.norm(b1)
b2_mag = linalg.norm(b2)
b3_mag = linalg.norm(b3)
beta1 = np.arccos(np.dot(b3, b2) / (linalg.norm(b3) * linalg.norm(b2)))
beta2 = np.arccos(np.dot(b1, b3) / (linalg.norm(b1) * linalg.norm(b3)))
beta3 = np.arccos(np.dot(b1, b2) / (linalg.norm(b1) * linalg.norm(b2)))

In [948]:
np.degrees(beta3)

60.254735809135845

In [949]:
a1 = (2 * np.pi * np.cross(b2, b3)) / (b1 @ (np.cross(b2, b3)))
a2 = (2 * np.pi * np.cross(b3, b1)) / (b1 @ (np.cross(b2, b3)))
a3 = (2 * np.pi * np.cross(b1, b2)) / (b1 @ (np.cross(b2, b3)))
V = a1 @ (np.cross(a2, a3))
a1_mag = linalg.norm(a1)
a2_mag = linalg.norm(a2)
a3_mag = linalg.norm(a3)
alpha1 = np.arccos(np.dot(a3, a2) / (linalg.norm(a3) * linalg.norm(a2)))
alpha2 = np.arccos(np.dot(a1, a3) / (linalg.norm(a1) * linalg.norm(a3)))
alpha3 = np.arccos(np.dot(a1, a2) / (linalg.norm(a1) * linalg.norm(a2)))

In [950]:
print(linalg.norm(a1))
print(linalg.norm(a2))
print(linalg.norm(a3))
print(np.degrees(alpha1))
print(np.degrees(alpha2))
print(np.degrees(alpha3))

4.752193877661791
4.745106056485653
12.97143911150735
90.31147516710972
89.88224041347546
119.74544856208159


In [951]:
a1_ref = test.phases['corundum'].a
a2_ref = test.phases['corundum'].b
a3_ref = test.phases['corundum'].c
V_ref = test.phases['corundum'].lattice.UnitCellVolume()
alpha1_ref = np.radians(test.phases['corundum'].alpha)
alpha2_ref = np.radians(test.phases['corundum'].beta)
alpha3_ref = np.radians(test.phases['corundum'].gamma)
b1_ref, b2_ref, b3_ref = test.phases['corundum'].B.T # The transpose bothers me
beta1_ref = np.radians(vector_angle(b2_ref, b3_ref))
b3_ref = linalg.norm(b3_ref)
rec_scale = 2 * np.pi
#rec_scale = 1

I = np.asarray([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

A_ref = np.array([[a1_ref, a2_ref * np.cos(alpha3_ref), a3_ref * np.cos(alpha2_ref)],
                  [0, a2_ref * np.sin(alpha3_ref), -a3_ref * np.sin(alpha2_ref) * np.cos(beta1_ref)],
                  [0, 0, rec_scale / b3_ref]])

delta = (-V - V_ref) / V_ref # negative because I messed up the coordinate system somehow...

A_meas = np.array([[a1_mag, a2_mag * np.cos(alpha3), a3_mag * np.cos(alpha2)],
                   [0, a2_mag * np.sin(alpha3), -a3_mag * np.sin(alpha2) * np.cos(beta1)],
                   [0, 0, rec_scale / b3_mag]])
#A_meas -= I * delta

#Tij = A_meas @ linalg.inv(A_ref)
Tij = linalg.inv(A_ref) @ A_meas
eij = 0.5 * (Tij + Tij.T) - I

In [775]:
eij * 1e3

array([[-1.01032633,  2.55860732, -1.20190563],
       [ 2.55860732,  0.0484595 , -8.00820118],
       [-1.20190563, -8.00820118, -1.26705309]])

In [846]:
A_ref[:, 0]

array([4.757, 0.   , 0.   ])

In [848]:
a1, a2, a3 = A_ref.T
a1

array([4.757, 0.   , 0.   ])

In [777]:
a0 = test.phases['corundum'].a
b0 = test.phases['corundum'].b
c0 = test.phases['corundum'].c
alpha0 = np.radians(test.phases['corundum'].alpha)
beta0 = np.radians(test.phases['corundum'].beta)
gamma0 = np.radians(test.phases['corundum'].gamma)
gamma0_rec = np.radians(vector_angle(*test.phases['corundum'].B[:, :2].T))

a1 = a1_mag
b1 = a2_mag
c1 = a3_mag
alpha1 = alpha1
beta1 = alpha2
gamma1 = alpha3
gamma1_rec = beta3

l11 = ((a1 * np.sin(beta1) * np.sin(gamma1_rec)) / (a0 * np.sin(beta0) * np.sin(gamma0_rec))) - 1
l22 = ((b1 * np.sin(alpha1)) / (b0 * np.sin(alpha0))) - 1
l33 = (c1 / c0) - 1

l12 = 0.5 * ((b1 * np.sin(alpha1) * np.cos(gamma0_rec)) / (b0 * np.sin(alpha0) * np.sin(gamma0_rec))
             - (a1 * np.sin(beta1) * np.cos(gamma1_rec)) / (a0 * np.sin(beta0) * np.sin(gamma0_rec)))

l13 = 0.5 * ((a1 * np.cos(beta1)) / (a0 * np.sin(beta0) * np.sin(gamma0_rec))
             + ((np.cos(gamma0_rec)) / (np.sin(gamma0_rec))) * ((b1 * np.cos(alpha1)) / (b0 * np.sin(alpha0))
                                                                - (c1 * np.cos(alpha0)) / (c0 * np.sin(alpha0)))
             - (c1 * np.cos(beta0)) / (c0 * np.sin(beta0) * np.sin(gamma0_rec)))

l23 = 0.5 * ((b1 * np.cos(alpha1)) / (b0 * np.sin(alpha0)) - (c1 * np.cos(alpha0)) / (c0 * np.sin(alpha0)))

strain = np.array([[l11, l12, l13],
                   [l12, l22, l23],
                   [l13, l23, l33]])
I = np.asarray([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [778]:
strain * 1e3

array([[ 1.54197003,  1.78981635, -0.37995972],
       [ 1.78981635, -2.51504297, -2.71132409],
       [-0.37995972, -2.71132409, -1.25202218]])

In [829]:
test_hkls = [[3, 1, 1], [2, 1, 0], [3, 2, 0]]
test_q = []

test_q.append(test.phases['corundum'].Q(test_hkls[0]) * 0.999)
test_q.append(test.phases['corundum'].Q(test_hkls[1]) * 1.001)
test_q.append(test.phases['corundum'].Q(test_hkls[2]) * 0.999)

In [830]:
x, res, rnk, s = linalg.lstsq(test_hkls, test_q)
x

array([[ 1.53888884,  0.00528332,  0.        ],
       [ 0.73894117,  1.31158356, -0.        ],
       [-0.02287744, -0.00792498,  0.4832959 ]])

In [831]:
b1, b2, b3 = x
b1_mag = linalg.norm(b1)
b2_mag = linalg.norm(b2)
b3_mag = linalg.norm(b3)
beta1 = np.arccos(np.dot(b3, b2) / (linalg.norm(b3) * linalg.norm(b2)))
beta2 = np.arccos(np.dot(b1, b3) / (linalg.norm(b1) * linalg.norm(b3)))
beta3 = np.arccos(np.dot(b1, b2) / (linalg.norm(b1) * linalg.norm(b2)))

a1 = (2 * np.pi * np.cross(b2, b3)) / (b1 @ (np.cross(b2, b3)))
a2 = (2 * np.pi * np.cross(b3, b1)) / (b1 @ (np.cross(b2, b3)))
a3 = (2 * np.pi * np.cross(b1, b2)) / (b1 @ (np.cross(b2, b3)))
V = a1 @ (np.cross(a2, a3))
a1_mag = linalg.norm(a1)
a2_mag = linalg.norm(a2)
a3_mag = linalg.norm(a3)
alpha1 = np.arccos(np.dot(a3, a2) / (linalg.norm(a3) * linalg.norm(a2)))
alpha2 = np.arccos(np.dot(a1, a3) / (linalg.norm(a1) * linalg.norm(a3)))
alpha3 = np.arccos(np.dot(a1, a2) / (linalg.norm(a1) * linalg.norm(a2)))

In [832]:
a1_ref = test.phases['corundum'].a
a2_ref = test.phases['corundum'].b
a3_ref = test.phases['corundum'].c
V_ref = test.phases['corundum'].lattice.UnitCellVolume()
alpha1_ref = np.radians(test.phases['corundum'].alpha)
alpha2_ref = np.radians(test.phases['corundum'].beta)
alpha3_ref = np.radians(test.phases['corundum'].gamma)
b1_ref, b2_ref, b3_ref = test.phases['corundum'].B.T # The transpose bothers me
beta1_ref = np.radians(vector_angle(b2_ref, b3_ref))
b3_ref = linalg.norm(b3_ref)
rec_scale = 2 * np.pi
#rec_scale = 1

I = np.asarray([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

A_ref = np.array([[a1_ref, a2_ref * np.cos(alpha3_ref), a3_ref * np.cos(alpha2_ref)],
                  [0, a2_ref * np.sin(alpha3_ref), -a3_ref * np.sin(alpha2_ref) * np.cos(beta1_ref)],
                  [0, 0, rec_scale / b3_ref]])

delta = (-V - V_ref) / V_ref # negative because I messed up the coordinate system somehow...

A_meas = np.array([[a1_mag, a2_mag * np.cos(alpha3), a3_mag * np.cos(alpha2)],
                   [0, a2_mag * np.sin(alpha3), -a3_mag * np.sin(alpha2) * np.cos(beta1)],
                   [0, 0, rec_scale / b3_mag]])
#A_meas -= I * delta

#Tij = A_meas @ linalg.inv(A_ref)
Tij = linalg.inv(A_ref) @ A_meas
eij = 0.5 * (Tij + Tij.T) - I

In [833]:
a0 = test.phases['corundum'].a
b0 = test.phases['corundum'].b
c0 = test.phases['corundum'].c
alpha0 = np.radians(test.phases['corundum'].alpha)
beta0 = np.radians(test.phases['corundum'].beta)
gamma0 = np.radians(test.phases['corundum'].gamma)
gamma0_rec = np.radians(vector_angle(*test.phases['corundum'].B[:, :2].T))

a1 = a1_mag
b1 = a2_mag
c1 = a3_mag
alpha1 = alpha1
beta1 = alpha2
gamma1 = alpha3
gamma1_rec = beta3

l11 = ((a1 * np.sin(beta1) * np.sin(gamma1_rec)) / (a0 * np.sin(beta0) * np.sin(gamma0_rec))) - 1
l22 = ((b1 * np.sin(alpha1)) / (b0 * np.sin(alpha0))) - 1
l33 = (c1 / c0) - 1

l12 = 0.5 * ((b1 * np.sin(alpha1) * np.cos(gamma0_rec)) / (b0 * np.sin(alpha0) * np.sin(gamma0_rec))
             - (a1 * np.sin(beta1) * np.cos(gamma1_rec)) / (a0 * np.sin(beta0) * np.sin(gamma0_rec)))

l13 = 0.5 * ((a1 * np.cos(beta1)) / (a0 * np.sin(beta0) * np.sin(gamma0_rec))
             + ((np.cos(gamma0_rec)) / (np.sin(gamma0_rec))) * ((b1 * np.cos(alpha1)) / (b0 * np.sin(alpha0))
                                                                - (c1 * np.cos(alpha0)) / (c0 * np.sin(alpha0)))
             - (c1 * np.cos(beta0)) / (c0 * np.sin(beta0) * np.sin(gamma0_rec)))

l23 = 0.5 * ((b1 * np.cos(alpha1)) / (b0 * np.sin(alpha0)) - (c1 * np.cos(alpha0)) / (c0 * np.sin(alpha0)))

strain = np.array([[l11, l12, l13],
                   [l12, l22, l23],
                   [l13, l23, l33]])
I = np.asarray([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [1283]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

for i in range(len(q_image_list)):
    int_mask = q_image_list[i][-1] > 1
    ax.scatter(*q_image_list[i][:3, int_mask],
               c=q_image_list[i][-1, int_mask], s=1, alpha=0.5)
    
    #ax.scatter(*q_image_list[i][:3],
    #           c=q_image_list[i][-1], s=1, alpha=0.5)

q_image_list[i][:3, int_mask]

ax.set_xlim(qx_min, qx_max)
ax.set_ylim(qy_min, qy_max)
ax.set_zlim(qz_min, qz_max)

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [ ]:
from scipy.interpolate import griddata

all_qx = []
all_qy = []
all_qz = []
all_int = []

for q_image in q_image_list:
    all_qx.extend(q_image[0])
    all_qy.extend(q_image[1])
    all_qz.extend(q_image[2])
    all_int.extend(q_image[3])

# Find bounds
x_min = np.min(all_qx)
x_max = np.max(all_qx)
y_min = np.min(all_qy)
y_max = np.max(all_qy)
z_min = np.min(all_qz)
z_max = np.max(all_qz)

# Generate q-space grid
gridstep = 0.001
xx = np.linspace(x_min, x_max, int((x_max - x_min) / gridstep))
yy = np.linspace(y_min, y_max, int((y_max - y_min) / gridstep))
zz = np.linspace(z_min, z_max, int((z_max - z_min) / gridstep))

grid = np.array(np.meshgrid(xx, yy, zz))
grid = grid.reshape(3, grid.size // 3).T

points = np.array([all_qx, all_qy, all_qz]).T

int_grid = griddata(points, all_int, grid, method='nearest')
int_grid = int_grid.reshape(yy.shape[0], xx.shape[0], zz.shape[0]).T

In [ ]:
from scipy.interpolate import griddata

all_qx = q_spot[0]
all_qy = q_spot[1]
all_qz = q_spot[2]
all_int = q_spot[3]

# Find bounds
x_min = np.min(all_qx)
x_max = np.max(all_qx)
y_min = np.min(all_qy)
y_max = np.max(all_qy)
z_min = np.min(all_qz)
z_max = np.max(all_qz)

# Generate q-space grid
gridstep = 0.001
xx = np.linspace(x_min, x_max, int((x_max - x_min) / gridstep))
yy = np.linspace(y_min, y_max, int((y_max - y_min) / gridstep))
zz = np.linspace(z_min, z_max, int((z_max - z_min) / gridstep))

grid = np.array(np.meshgrid(xx, yy, zz))
grid = grid.reshape(3, grid.size // 3).T

points = np.array([all_qx, all_qy, all_qz]).T

int_grid = griddata(points, all_int, grid, method='nearest')
int_grid = int_grid.reshape(yy.shape[0], xx.shape[0], zz.shape[0]).T

In [ ]:
from skimage import measure
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

spacing = 0.005

vol = int_grid
#vol = GaussianFunctions.func_3d(grid, *popt).reshape(*int_grid.shape)
#vol = GaussianFunctions.func_3d(np.array(np.meshgrid(xx, yy, zz)), *popt).T.reshape(*int_grid.shape)

iso_vals = np.linspace(np.min(int_grid), np.max(int_grid), 7)[1:-1]
iso_vals = [1, 5, 10, 20, 40, 80]

for i, iso_val in enumerate(iso_vals):

    #vol = int_grid
    #iso_val = 10000
    #spacing = 0.001
    color = cm.viridis(i / len(iso_vals))
    verts, faces, _, _ = measure.marching_cubes(vol, iso_val)

    # I have no idea how the axes got so mixed up...
    plot_x = np.interp(verts[:, 1], range(len(xx)), xx)
    plot_y = np.interp(verts[:, 2], range(len(yy)), yy)
    plot_z = np.interp(verts[:, 0], range(len(zz)), zz)

    ax.plot_trisurf(plot_x, plot_y, plot_z, triangles=faces, alpha=0.1, color=color)

ax.set_xlim(np.min(all_qx), np.max(all_qx))
ax.set_ylim(np.min(all_qy), np.max(all_qy))
ax.set_zlim(np.min(all_qz), np.max(all_qz))

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [ ]:
np.fft.fftn(int_grid).shape

(99, 99, 96)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

plot_grid = np.log(np.abs(np.fft.fftshift(np.fft.fftn(int_grid)))**2)

ax.scatter(plot_grid[0].ravel(),
           plot_grid[1].ravel(),
           plot_grid[2].ravel(),
           s=1, alpha=0.1)

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

plot_grid = np.log(np.abs(np.fft.fftshift(np.fft.fftn(int_grid[65])))**2)

im = ax.imshow(plot_grid)
fig.colorbar(im, ax=ax)

plt.show()

In [ ]:
plt.close('all')

In [ ]:
all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['zincite'], tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

ax.scatter(*np.asarray(all_qs).T, c='k', s=all_fs/20)
ax.scatter(0, 0, 0, c='k', s=10)

'''# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')'''

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()

In [ ]:
silicon.Q(1, 1, 1)

array([1.15697522, 1.15697522, 1.15697522])

In [ ]:
e11, e12, e13, e22, e23, e33 = 1e-6, 2e-6, -3e-6, 1e-6, -3e-6, 2e-6
w1, w2, w3 = 3e-5, 5e-5, 1e-4

eij = np.asarray([[e11, e12, e13], [e12, e22, e23], [e13, e23, e33]])
wij = np.asarray([[0, -w3, w2], [w3, 0, -w1], [-w2, w1, 0]])
I = np.asarray([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

duij = eij + I

In [ ]:
phi_x, phi_y, phi_z = 30, 40, 60

phi_x, phi_y, phi_z = np.radians([phi_x, phi_y, phi_z])

Rx = np.asarray([[1, 0, 0],
                 [0, np.cos(phi_x), -np.sin(phi_x)],
                 [0, np.sin(phi_x), np.cos(phi_x)]])

Ry = np.asarray([[np.cos(phi_y), 0, np.sin(phi_y)],
                 [0, 1, 0],
                 [-np.sin(phi_y), 0, np.cos(phi_y)]])

Rz = np.asarray([[np.cos(phi_z), -np.sin(phi_z), 0],
                 [np.sin(phi_z), np.cos(phi_z), 0],
                 [0, 0, 1]])

rot = Rx @ Ry @ Rz

Fij =  duij @ rot

In [ ]:
Fij

array([[ 0.38302396, -0.6634165 ,  0.6427855 ],
       [ 0.91069812,  0.15467413, -0.38302331],
       [ 0.15467393,  0.7320937 ,  0.6634145 ]])

In [ ]:
rot

array([[ 0.38302222, -0.66341395,  0.64278761],
       [ 0.9106969 ,  0.1546775 , -0.38302222],
       [ 0.1546775 ,  0.73209071,  0.66341395]])

In [ ]:
from scipy.linalg import polar

In [ ]:
p, u = linalg.polar(Fij, side='left')
print(u)
print(p)

[[ 1.000001  0.000002 -0.000003]
 [ 0.000002  1.000001 -0.000003]
 [-0.000003 -0.000003  1.000002]]
[[ 0.38302222 -0.66341395  0.64278761]
 [ 0.9106969   0.1546775  -0.38302222]
 [ 0.1546775   0.73209071  0.66341395]]


In [ ]:
print(u - duij)

[[ 0.  0.  0.]
 [ 0.  0. -0.]
 [ 0.  0.  0.]]


In [ ]:
u

array([[ 1.000001,  0.000002, -0.000003],
       [ 0.000002,  1.000001, -0.000003],
       [-0.000003, -0.000003,  1.000002]])

In [ ]:
rec_duij = u - I
rec_eij = np.round(0.5 * (rec_duij.T + rec_duij), 10)
rec_wij = np.round(0.5 * (rec_duij - rec_duij.T), 10)

In [ ]:
rec_e11 = rec_eij[0, 0]
rec_e12 = rec_eij[0, 1]
rec_e13 = rec_eij[0, 2]
rec_e22 = rec_eij[1, 1]
rec_e23 = rec_eij[1, 2]
rec_e33 = rec_eij[2, 2]

In [ ]:
print(f'ε₁₁ is {rec_e11 * 1e6} µε.')
print(f'ε₂₂ is {rec_e22 * 1e6} µε.')
print(f'ε₃₃ is {rec_e33 * 1e6} µε.')
print(f'ε₁₂ is {rec_e12 * 1e6} µε.')
print(f'ε₁₃ is {rec_e13 * 1e6} µε.')
print(f'ε₂₃ is {rec_e23 * 1e6} µε.')

ε₁₁ is 1.0 µε.
ε₂₂ is 1.0 µε.
ε₃₃ is 2.0 µε.
ε₁₂ is 2.0 µε.
ε₁₃ is -3.0 µε.
ε₂₃ is -3.0 µε.
